# 1. Backpropagation for bias vectors (1 point)

In class, we discussed a multilayer perceptron (neural network) whose layers were all "dense", i.e. the output $a^m \in \mathbb{R}^{N^m}$ of the $m$th layer is computed as 
\begin{align*}
z^m &= W^m a^{m - 1} + b^m \\
a^m &= \sigma^m(z^m)
\end{align*}
where $W^m \in \mathbb{R}^{N^m \times N^{m - 1}}$ is the weight matrix, $b^m \in \mathbb{R}^{N^m}$ is the bias vector, and $\sigma^m$ is the nonlinearity. We showed that 
$$\frac{\partial C}{\partial W^m} = \frac{\partial C}{\partial z^m} a^{m - 1 \top}$$
Show that
$$\frac{\partial C}{\partial b^m} = \frac{\partial C}{\partial z^m}$$
Hint: The derivation is almost the same as for $W$.

Proof: $$\frac{\partial C}{\partial b^m} = \sum_{k=1}^{N}\frac{\partial C}{\partial z^m_{k}} \frac{\partial z^m_{k}}{\partial b^m_{ij}}$$

When i=k 
\begin{align*}
\frac{\partial z_{i}^{m}}{\partial b^m_{ij}} = 
\frac{\partial}{\partial b^m_{ij}} (\sum_{l=1}^{N}W_{il}^{m} a_{l}^{m-1}+b_i^m)=\vec1
\end{align*}

Otherwise, $$\frac{\partial z_{i}^{m}}{\partial b^m_{ij}} =0$$

Thus, $$\frac{\partial C}{\partial b^m_{ij}} = \frac{\partial C}{\partial z^m_{i}} \cdot \vec1$$

Using the matrix notation, $$\frac{\partial C}{\partial b^m} = \frac{\partial C}{\partial z^m}$$

# 2. MLP from scratch (3 points)

Using numpy only, implement backward pass or a sigmoid MLP. Specifically, you will need to implement this functionality in the `train` function in the `SigmoidMLP` class below. You should write numpy code to populate the two lists `weight_gradients` and `bias_gradient`, where each entry in each list corresponds to the gradient for a weight matrix or bias vector for each layer. Then, when you run the code cell at the bottom of this notebook, the trained MLP should output (approximately) 0, 1, 1, 0, having learned the [XOR function](https://en.wikipedia.org/wiki/Exclusive_or). Please us a binary cross-entropy loss, i.e.
$$C(a^L, y) = (y - 1)\log(1 - a^L) - y\log(a^L)$$

*Note 1*: All layers in your model, including the last layer, will use the sigmoid cross-entropy function. Remember that 
$$
\frac{\partial}{\partial x}\mathrm{sigmoid}(x) = \mathrm{sigmoid}(x)(1 - \mathrm{sigmoid}(x))$$

*Note 2*: As we mentioned in class,
$$
\frac{\partial C}{\partial z^L} = a^L - y
$$

In [ ]:
import numpy as np
def sigmoid(x):
  
    z = np.exp(-x)
    sig = 1 / (1 + z)

    return sig

In [ ]:

class Layer:
    def __init__(self, inputs, outputs):
        # Initialize weight matrix and bias vector
        # Getting the initialization right can be tricky, but for this problem
        # simply drawing from a standard normal distribution should work.
        self.weights = np.random.randn(outputs, inputs)
        self.biases = np.random.randn(outputs, 1)
    def __call__(self, X):
        # Compute \sigmoid(Wx + b)
        return 1/(1 + np.exp(-(self.weights.dot(X) + self.biases)))

class SigmoidMLP:

    def __init__(self, layer_widths):
        self.layers = []
        for inputs, outputs in zip(layer_widths[:-1], layer_widths[1:]):
            self.layers.append(Layer(inputs, outputs))
    
    def train(self, inputs, targets, learning_rate):
        # Forward pass - compute each layer's output and store it for later use
        layer_outputs = [inputs]
        for layer in self.layers:
            layer_outputs.append(layer(layer_outputs[-1]))
        
        # Implement backward pass to populate weight_gradients and bias_gradients
        # lists here
        weight_gradients = []
        bias_gradients = []
        # ... (your code here) ...
        a2 = layer_outputs[-1]
        a1 = layer_outputs[-2]
        a0 = layer_outputs[-3]
        W1 = self.layers[0].weights
        b1 = self.layers[0].biases
        Z1 = np.dot(W1,a0) + b1
        W2 = self.layers[1].weights
        cost = (targets-1)*np.log(1-a2) - targets*np.log(a2)
        da2 = -targets/a2 + (1-targets)/(1-a2)
        dZ2 = a2-targets
        dW2 = np.dot(dZ2,a1.T)
        db2 = np.sum(dZ2, axis = 1, keepdims = True)
        da1 = np.dot(W2.T,dZ2)
        dZ1 = da1*(sigmoid(Z1)*(1-sigmoid(Z1)))
        dW1 = np.dot(dZ1,a0.T)
        db1 = np.sum(dZ1, axis = 1, keepdims = True)
        weight_gradients.append(dW1)
        weight_gradients.append(dW2)
        bias_gradients.append(db1)
        bias_gradients.append(db2)

        # Perform gradient descent by applying updates
        for weight_gradient, bias_gradient, layer in zip(weight_gradients, bias_gradients, self.layers):
            layer.weights -= weight_gradient*learning_rate
            layer.biases -= bias_gradient*learning_rate

    def __call__(self, inputs):
        a = inputs
        for layer in self.layers:
            a = layer(a)
        return a

def train_mlp(n_iterations, learning_rate):
    mlp = SigmoidMLP([2, 2, 1])
    inputs = np.array([[0, 1, 0, 1], 
                       [0, 0, 1, 1]])
    targets = np.array([[0, 1, 1, 0]])
    for _ in range(int(1e3)):
        mlp.train(inputs, targets, learning_rate)
    return mlp

In [ ]:
# You may need to change the n_iterations and learning_rate values
# but these worked for me
mlp = train_mlp(1000, 1)
# The following calls should result in (approximately) 0, 1, 1, 0
# If the outputs are somewhat close, your training has succeeded!
print(mlp(np.array([0, 0]).reshape(-1, 1)))
print(mlp(np.array([0, 1]).reshape(-1, 1)))
print(mlp(np.array([1, 0]).reshape(-1, 1)))
print(mlp(np.array([1, 1]).reshape(-1, 1)))

[[0.00247338]]
[[0.99741727]]
[[0.99741766]]
[[0.00386548]]
